## IMPORTS

* Nome da semente, estacao da semente, quanto custa, quanto vende cada qualidade, quantos quadrados exige, quanto tempo leva (e se dá mais de 1x. Caso de, de quantos em quantos dias apos a primeira colheita- implementar isso dps)

In [22]:
!pip install unidecode


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
!pip install flask_sqlalchemy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!pip install flask


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
!pip install psycopg2


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import requests
from bs4 import BeautifulSoup
import re
# revisar os imports dps de finalizar. Ver quais nao estao sendo utilizados
from unidecode import unidecode
import pandas as pd

## Pegar o quadro dos frutos

In [28]:
valores_colunas_dataframe = ["Nome_Fruto", "Inverno", "Outono", "Primavera", "Verão", "Preco_Compra",
                             "Local_Compra", "Preco_Venda_Comum", "Preco_Venda_Incomum", "Dias_Para_Amadurecer", "Dias_Até_Colher_Novamente"]

In [29]:
dataframe_stardewvalley = pd.DataFrame(columns=valores_colunas_dataframe)

In [30]:
dataframe_stardewvalley.columns

Index(['Nome_Fruto', 'Inverno', 'Outono', 'Primavera', 'Verão', 'Preco_Compra',
       'Local_Compra', 'Preco_Venda_Comum', 'Preco_Venda_Incomum',
       'Dias_Para_Amadurecer', 'Dias_Até_Colher_Novamente'],
      dtype='object')

In [31]:
URL = "https://pt.stardewvalleywiki.com/Lavouras"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [32]:
results = soup.find()

In [33]:
frutos_quadros = results.find_all("table", class_="wikitable roundedborder")

In [34]:
len(frutos_quadros)

41

In [35]:
frutos_quadros.pop()

<table class="wikitable roundedborder">
<tbody><tr>
<th>Estação</th>
<th>Sementes
</th></tr>
<tr>
<th>Primavera
</th>
<td style="padding: 6px;"><link href="mw-data:TemplateStyles:r22059" rel="mw-deduplicated-inline-style"/><span class="nametemplateinline"><img alt="Cauliflower Seeds.png" decoding="async" height="24" src="https://stardewvalleywiki.com/mediawiki/images/thumb/b/bb/Cauliflower_Seeds.png/24px-Cauliflower_Seeds.png" srcset="https://stardewvalleywiki.com/mediawiki/images/thumb/b/bb/Cauliflower_Seeds.png/36px-Cauliflower_Seeds.png 1.5x, https://stardewvalleywiki.com/mediawiki/images/b/bb/Cauliflower_Seeds.png 2x" width="24"/> <a href="/Sementes_de_Couve-flor" title="Sementes de Couve-flor">Sementes de Couve-flor</a></span> • <link href="mw-data:TemplateStyles:r22059" rel="mw-deduplicated-inline-style"/><span class="nametemplateinline"><img alt="Parsnip Seeds.png" decoding="async" height="24" src="https://stardewvalleywiki.com/mediawiki/images/thumb/d/d3/Parsnip_Seeds.png/24px-

In [36]:
len(frutos_quadros)

40

## Pegar nome do fruto

In [37]:
padrao_nomes_frutos = re.compile(r'\s(.*)')

In [38]:
h3_nomes_frutos = results.find_all("h3")

In [39]:
lista_nomes_frutos = []

for h3_nome_fruto in h3_nomes_frutos:

  nomes_frutos = h3_nome_fruto.find_all("a")

  for nome_fruto in nomes_frutos:

    if nome_fruto.text.strip() and len(lista_nomes_frutos) < 40:

      lista_nomes_frutos.append(nome_fruto.text)

In [40]:
dataframe_stardewvalley["Nome_Fruto"] = lista_nomes_frutos

In [41]:
dataframe_stardewvalley["Nome_Fruto"].isna().sum()

0

In [42]:
dataframe_stardewvalley.shape

(40, 11)

In [43]:
dataframe_stardewvalley.tail()

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
35,Cereja de Joia Doce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Fruta antiga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Fruto do cacto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,Abacaxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,Folhas de Chá,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
dataframe_stardewvalley = dataframe_stardewvalley.drop(
    dataframe_stardewvalley.tail(5).index)

In [45]:
dataframe_stardewvalley.shape

(35, 11)

## Pegar preco e local de venda do fruto

In [46]:
padrao_valores_vendas = re.compile(r'(?:Armazém|Mercado Joja).*')

In [47]:
# lista dos valores comuns, pois a formatacao das sementes especiais é diferente
lista_menores_valores_venda_comuns = []

for fruto_quadro in frutos_quadros:  # para cada semente, pegar apenas o local que vende mais barato

  preco_compra_local_venda = fruto_quadro.find_all("p")

  for preco_local in preco_compra_local_venda:

    valores_vendas = re.sub(r'data-sort-value="[^"]+">', '', preco_local.text)

    if 'ouros' in valores_vendas and '/d' not in valores_vendas:

      valores_vendas_splitlines = valores_vendas.splitlines()

      menor_valor_venda = valores_vendas_splitlines[0]

      if len(valores_vendas_splitlines) > 0:

        menor_valor_venda = valores_vendas_splitlines[0]

        for valor_venda in valores_vendas_splitlines[1:]:

          if int(re.search(r'(\d+)', menor_valor_venda).group()) > int(re.search(r'(\d+)', valor_venda).group()) and 'drop' not in valor_venda:

            menor_valor_venda = valor_venda

      lista_menores_valores_venda_comuns.append(
        re.sub(r'\xa0ouros', '', menor_valor_venda))

In [48]:
len(lista_menores_valores_venda_comuns)

34

In [49]:
lista_menores_valores_venda_comuns

['Armazém 40',
 'Armazém 50',
 'Armazém 20',
 'Armazém 70',
 'Armazém 80',
 'Carrinho de Viagem 2\xa0500',
 'Armazém 30',
 'Festival do Ovo 100',
 'Oásis 100',
 'Armazém 20',
 'Armazém 60',
 'Oásis 400',
 'Armazém 50',
 'Mercado Joja 125',
 'Armazém 60',
 'Armazém 80',
 'Armazém 150',
 'Armazém 80',
 'Armazém 100',
 'Armazém 40',
 'Armazém 40',
 'Armazém 100',
 'Armazém 50',
 'Armazém 10',
 'Armazém 100',
 'Armazém 30',
 'Armazém 70',
 'Armazém 20',
 'Oásis 20',
 'Armazém 50',
 'Armazém 60',
 'Armazém 240',
 'Armazém 200',
 'Armazém 60']

In [50]:
lista_menores_valores_venda_comuns[5] = re.sub(
    r'\xa0', '', lista_menores_valores_venda_comuns[5])

In [51]:
lista_menores_valores_venda_comuns

['Armazém 40',
 'Armazém 50',
 'Armazém 20',
 'Armazém 70',
 'Armazém 80',
 'Carrinho de Viagem 2500',
 'Armazém 30',
 'Festival do Ovo 100',
 'Oásis 100',
 'Armazém 20',
 'Armazém 60',
 'Oásis 400',
 'Armazém 50',
 'Mercado Joja 125',
 'Armazém 60',
 'Armazém 80',
 'Armazém 150',
 'Armazém 80',
 'Armazém 100',
 'Armazém 40',
 'Armazém 40',
 'Armazém 100',
 'Armazém 50',
 'Armazém 10',
 'Armazém 100',
 'Armazém 30',
 'Armazém 70',
 'Armazém 20',
 'Oásis 20',
 'Armazém 50',
 'Armazém 60',
 'Armazém 240',
 'Armazém 200',
 'Armazém 60']

In [52]:
valor_menor_valor_venda_arroz_nao_moido = ""

for fruto_quadro in frutos_quadros:

  divs = fruto_quadro.find_all("div", class_="no-wrap")

  for div in divs:

    valor_arroz_nao_moido = re.sub(
      r'.*?(Armazém): data-sort-value="(\d+).*', r'\1 \2', div.text)

    if ":" not in valor_arroz_nao_moido:

      lista_menores_valores_venda_comuns.insert(1, valor_arroz_nao_moido)

In [53]:
len(lista_menores_valores_venda_comuns)

35

In [54]:
lista_menores_valores_venda_comuns

['Armazém 40',
 'Armazém 40',
 'Armazém 50',
 'Armazém 20',
 'Armazém 70',
 'Armazém 80',
 'Carrinho de Viagem 2500',
 'Armazém 30',
 'Festival do Ovo 100',
 'Oásis 100',
 'Armazém 20',
 'Armazém 60',
 'Oásis 400',
 'Armazém 50',
 'Mercado Joja 125',
 'Armazém 60',
 'Armazém 80',
 'Armazém 150',
 'Armazém 80',
 'Armazém 100',
 'Armazém 40',
 'Armazém 40',
 'Armazém 100',
 'Armazém 50',
 'Armazém 10',
 'Armazém 100',
 'Armazém 30',
 'Armazém 70',
 'Armazém 20',
 'Oásis 20',
 'Armazém 50',
 'Armazém 60',
 'Armazém 240',
 'Armazém 200',
 'Armazém 60']

In [55]:
lista_preco_compra = [item.split()[-1]
                      for item in lista_menores_valores_venda_comuns]
lista_local_compra = [item.split()[0]
                      for item in lista_menores_valores_venda_comuns]

In [56]:
len(lista_preco_compra)

35

In [57]:
len(lista_local_compra)

35

In [58]:
dataframe_stardewvalley["Preco_Compra"] = lista_preco_compra

In [59]:
dataframe_stardewvalley["Local_Compra"] = lista_local_compra

In [60]:
dataframe_stardewvalley.head()

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
0,Alho,NaN,NaN,NaN,NaN,40,Armazém,NaN,NaN,NaN,NaN
1,Arroz não moído,NaN,NaN,NaN,NaN,40,Armazém,NaN,NaN,NaN,NaN
2,Batata,NaN,NaN,NaN,NaN,50,Armazém,NaN,NaN,NaN,NaN
3,Chirívia,NaN,NaN,NaN,NaN,20,Armazém,NaN,NaN,NaN,NaN
4,Couve,NaN,NaN,NaN,NaN,70,Armazém,NaN,NaN,NaN,NaN


In [61]:
dataframe_stardewvalley[["Preco_Compra", "Local_Compra"]].isna().sum()

Preco_Compra    0
Local_Compra    0
dtype: int64

## Pegar preco venda comum, preco venda incomum e estações de crescimento

In [62]:
estacoes = ["Inverno", "Outono", "Primavera", "Verão"]

lista_valor_venda_comum = []

lista_valor_venda_incomum = []

lista_valor_booleano_estacoes = []

# pegar estacao aq tbm
for nome_busca_fruto in dataframe_stardewvalley["Nome_Fruto"]:

  lista_elemento_valor_booleano_estacoes = [0, 0, 0, 0]

  URL_sementes_especificas = f"https://pt.stardewvalleywiki.com/{
    nome_busca_fruto.replace(' ', '_')}"

  print(URL_sementes_especificas)

  page_sementes_especificas = requests.get(URL_sementes_especificas)

  soup_sementes_especificas = BeautifulSoup(
    page_sementes_especificas.content, "html.parser")

  results_sementes_especificas = soup_sementes_especificas.find()

  tabela_valores_venda = results_sementes_especificas.find(
    "table", style="width: 101%; margin-top: -2px; margin-left: -2px;")

  if tabela_valores_venda:

    tds_valor_venda = tabela_valores_venda.find_all("td")

  else:

    tds_valor_venda = results_sementes_especificas.find(
      "div", id="infoboxborder")

  for td_valor_venda in tds_valor_venda:

    numeros = re.findall(r'\b(\d+)\souros\b', td_valor_venda.text)

    if numeros:

      lista_valor_venda_comum.append(numeros[0])

      lista_valor_venda_incomum.append(numeros[1])

      break

  div_estacoes_crescimento = results_sementes_especificas.find(
    "div", id="infoboxborder")

  tds_estacoes_crescimento = div_estacoes_crescimento.find_all(
    "td", id="infoboxdetail")

  for td_estacoes_crescimento in tds_estacoes_crescimento:

    for i, estacao in enumerate(estacoes):

      if estacao in td_estacoes_crescimento.text:

        lista_elemento_valor_booleano_estacoes[i] = 1

  lista_valor_booleano_estacoes.append(lista_elemento_valor_booleano_estacoes)

https://pt.stardewvalleywiki.com/Alho
https://pt.stardewvalleywiki.com/Arroz_não_moído
https://pt.stardewvalleywiki.com/Batata
https://pt.stardewvalleywiki.com/Chirívia
https://pt.stardewvalleywiki.com/Couve
https://pt.stardewvalleywiki.com/Couve-flor
https://pt.stardewvalleywiki.com/Grão_de_café
https://pt.stardewvalleywiki.com/Jasmim-Azul
https://pt.stardewvalleywiki.com/Morango
https://pt.stardewvalleywiki.com/Ruibarbo
https://pt.stardewvalleywiki.com/Tulipa
https://pt.stardewvalleywiki.com/Vagem
https://pt.stardewvalleywiki.com/Carambola
https://pt.stardewvalleywiki.com/Flor-Miçanga
https://pt.stardewvalleywiki.com/Girassol
https://pt.stardewvalleywiki.com/Lúpulo
https://pt.stardewvalleywiki.com/Melão
https://pt.stardewvalleywiki.com/Milho
https://pt.stardewvalleywiki.com/Mirtilo
https://pt.stardewvalleywiki.com/Papoula
https://pt.stardewvalleywiki.com/Pimenta_quente
https://pt.stardewvalleywiki.com/Rabanete
https://pt.stardewvalleywiki.com/Repolho_roxo
https://pt.stardewvalleywiki

In [63]:
len(lista_valor_booleano_estacoes)

35

In [64]:
len(lista_valor_venda_comum)

35

In [65]:
len(lista_valor_venda_incomum)

35

In [66]:
lista_valor_booleano_estacoes_bool = []
listinha = []
for lista in lista_valor_booleano_estacoes:
  listinha = []
  for val in lista:
    val_bool = bool(val)
    listinha.append(val_bool)
    if len(listinha) == 4:
      lista_valor_booleano_estacoes_bool.append(listinha)

In [67]:
lista_valor_booleano_estacoes

[[0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 1, 0, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 1]]

In [68]:
lista_valor_booleano_estacoes_bool

[[False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, True],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, True, False],
 [False, False, False, True],
 [False, False, False, True],
 [False, True, False, True],
 [False, False, False, True],
 [False, False, False, True],
 [False, True, False, True],
 [False, False, False, True],
 [False, False, False, True],
 [False, False, False, True],
 [False, False, False, True],
 [False, False, False, True],
 [False, False, False, True],
 [False, True, False, True],
 [False, True, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, True, False, False],
 [False, True,

In [69]:
for i, linha in enumerate(lista_valor_booleano_estacoes):
  dataframe_stardewvalley.loc[i, ["Inverno",
                                  "Outono", "Primavera", "Verão"]] = linha

In [70]:
dataframe_stardewvalley.head()

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
0,Alho,0,0,1,0,40,Armazém,NaN,NaN,NaN,NaN
1,Arroz não moído,0,0,1,0,40,Armazém,NaN,NaN,NaN,NaN
2,Batata,0,0,1,0,50,Armazém,NaN,NaN,NaN,NaN
3,Chirívia,0,0,1,0,20,Armazém,NaN,NaN,NaN,NaN
4,Couve,0,0,1,0,70,Armazém,NaN,NaN,NaN,NaN


In [71]:
dataframe_stardewvalley[["Inverno", "Outono",
                         "Primavera", "Verão"]].isna().sum()

Inverno      0
Outono       0
Primavera    0
Verão        0
dtype: int64

In [72]:
lista_valor_venda_comum

['60',
 '30',
 '80',
 '35',
 '110',
 '175',
 '15',
 '50',
 '120',
 '220',
 '30',
 '40',
 '750',
 '90',
 '80',
 '25',
 '250',
 '50',
 '50',
 '140',
 '40',
 '90',
 '260',
 '60',
 '25',
 '320',
 '160',
 '150',
 '60',
 '100',
 '80',
 '160',
 '75',
 '290',
 '80']

In [73]:
dataframe_stardewvalley["Preco_Venda_Comum"] = lista_valor_venda_comum

In [74]:
dataframe_stardewvalley["Preco_Venda_Comum"].isna().sum()

0

In [75]:
lista_valor_venda_incomum

['75',
 '37',
 '100',
 '43',
 '137',
 '218',
 '18',
 '62',
 '150',
 '275',
 '37',
 '50',
 '937',
 '112',
 '100',
 '31',
 '312',
 '62',
 '62',
 '175',
 '50',
 '112',
 '325',
 '75',
 '31',
 '400',
 '200',
 '187',
 '75',
 '125',
 '100',
 '200',
 '93',
 '362',
 '100']

In [76]:
dataframe_stardewvalley["Preco_Venda_Incomum"] = lista_valor_venda_incomum

In [77]:
dataframe_stardewvalley["Preco_Venda_Incomum"].isna().sum()

0

In [78]:
dataframe_stardewvalley.head()

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
0,Alho,0,0,1,0,40,Armazém,60,75,NaN,NaN
1,Arroz não moído,0,0,1,0,40,Armazém,30,37,NaN,NaN
2,Batata,0,0,1,0,50,Armazém,80,100,NaN,NaN
3,Chirívia,0,0,1,0,20,Armazém,35,43,NaN,NaN
4,Couve,0,0,1,0,70,Armazém,110,137,NaN,NaN


## Pegar quantidade de dias para amadurecer

In [79]:
padrao_dias_amadurecimento = r'Total: (\d+(-\d+)?\sDias)'

lista_dias_amadurecimento = []

i = 0

for fruto_quadro in frutos_quadros:

  dicionario_dias_amadurecimento = {}

  if i > 34:

    break

  infos = fruto_quadro.find_all("td")

  for info in infos:

    info_apenas_texto = info.get_text()

    matches = re.findall(padrao_dias_amadurecimento, info_apenas_texto)

    if matches:

      dias = matches[0][0]

      if '-' in dias:

        dias = dias.split('-')[1]

      dicionario_dias_amadurecimento[i] = int(
        re.search(r'(\d+)', dias).group())

      lista_dias_amadurecimento.append(dicionario_dias_amadurecimento)

  i += 1

In [80]:
lista_dias_amadurecimento

[{0: 4},
 {1: 8},
 {2: 6},
 {3: 4},
 {4: 6},
 {5: 12},
 {6: 10},
 {7: 7},
 {8: 8},
 {9: 13},
 {10: 6},
 {11: 10},
 {12: 13},
 {13: 8},
 {14: 8},
 {15: 11},
 {16: 12},
 {17: 14},
 {18: 13},
 {19: 7},
 {20: 5},
 {21: 6},
 {22: 9},
 {23: 11},
 {24: 4},
 {25: 13},
 {26: 8},
 {27: 7},
 {28: 5},
 {29: 6},
 {30: 4},
 {31: 10},
 {32: 7},
 {33: 12},
 {34: 10}]

In [81]:
for elementos in lista_dias_amadurecimento:
  dataframe_stardewvalley.loc[elementos.keys(
  ), 'Dias_Para_Amadurecer'] = list(elementos.values())[0]

In [82]:
dataframe_stardewvalley.head()

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
0,Alho,0,0,1,0,40,Armazém,60,75,4,NaN
1,Arroz não moído,0,0,1,0,40,Armazém,30,37,8,NaN
2,Batata,0,0,1,0,50,Armazém,80,100,6,NaN
3,Chirívia,0,0,1,0,20,Armazém,35,43,4,NaN
4,Couve,0,0,1,0,70,Armazém,110,137,6,NaN


In [83]:
dataframe_stardewvalley['Dias_Para_Amadurecer'].isna().sum()

0

## Pegar quantidade de dias até colher novamente

In [84]:
padrao_dias_colher_novamente = r'Continua aproduzir (a cada (\d+) dias|todo dia)'

lista_dias_colher_novamente = []

i = 0

for fruto_quadro in frutos_quadros:

  dicionario_dias_colher_novamente = {}

  if i > 34:

    break

  infos = fruto_quadro.find_all("td")

  for info in infos:

    info_apenas_texto = str(info.get_text())

    matches = re.findall(padrao_dias_colher_novamente, info_apenas_texto)

    if matches:

      if matches[0][1]:

        dias = matches[0][1]

      else:

        dias = '1' if matches[0][0] == 'todo dia' else matches[0][0]

      dicionario_dias_colher_novamente[i] = dias

      lista_dias_colher_novamente.append(dicionario_dias_colher_novamente)

  i += 1

print(lista_dias_colher_novamente)

[{6: '2'}, {8: '4'}, {11: '3'}, {15: '1'}, {17: '4'}, {18: '4'}, {20: '3'}, {23: '4'}, {28: '5'}, {32: '5'}, {34: '3'}]


In [85]:
for elementos in lista_dias_colher_novamente:
  dataframe_stardewvalley.loc[elementos.keys(
  ), 'Dias_Até_Colher_Novamente'] = list(elementos.values())[0]

In [86]:
dataframe_stardewvalley

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
0,Alho,0,0,1,0,40,Armazém,60,75,4,NaN
1,Arroz não moído,0,0,1,0,40,Armazém,30,37,8,NaN
2,Batata,0,0,1,0,50,Armazém,80,100,6,NaN
3,Chirívia,0,0,1,0,20,Armazém,35,43,4,NaN
4,Couve,0,0,1,0,70,Armazém,110,137,6,NaN
5,Couve-flor,0,0,1,0,80,Armazém,175,218,12,NaN
6,Grão de café,0,0,1,1,2500,Carrinho,15,18,10,2
7,Jasmim-Azul,0,0,1,0,30,Armazém,50,62,7,NaN
8,Morango,0,0,1,0,100,Festival,120,150,8,4
9,Ruibarbo,0,0,1,0,100,Oásis,220,275,13,NaN


## Converter as colunas para os tipos desejados

In [87]:
dataframe_stardewvalley.columns

Index(['Nome_Fruto', 'Inverno', 'Outono', 'Primavera', 'Verão', 'Preco_Compra',
       'Local_Compra', 'Preco_Venda_Comum', 'Preco_Venda_Incomum',
       'Dias_Para_Amadurecer', 'Dias_Até_Colher_Novamente'],
      dtype='object')

In [88]:
colunas_para_converter = ['Inverno', 'Outono', 'Primavera', 'Verão', 'Preco_Compra',
                          'Preco_Venda_Comum', 'Preco_Venda_Incomum', 'Dias_Para_Amadurecer', 'Dias_Até_Colher_Novamente']

In [89]:
dataframe_stardewvalley[colunas_para_converter] = dataframe_stardewvalley[colunas_para_converter].apply(
    pd.to_numeric, errors='coerce', downcast='integer')

In [90]:
dataframe_stardewvalley.dtypes

Nome_Fruto                    object
Inverno                         int8
Outono                          int8
Primavera                       int8
Verão                           int8
Preco_Compra                   int16
Local_Compra                  object
Preco_Venda_Comum              int16
Preco_Venda_Incomum            int16
Dias_Para_Amadurecer            int8
Dias_Até_Colher_Novamente    float64
dtype: object

## Algumas sementes retornam mais de 1 fruto por colheita

In [91]:
dicionario_multiplicador_colheita = {
    "Oxicoco": 2, "Girassol": 3, "Grão de café": 4}

In [92]:
def aplicar_multiplicador(row):

  nome_fruto = row["Nome_Fruto"]

  multiplicador = dicionario_multiplicador_colheita.get(nome_fruto, 1)

  row["Preco_Venda_Comum"] *= multiplicador

  row["Preco_Venda_Incomum"] *= multiplicador

  return row

In [93]:
dataframe_stardewvalley = dataframe_stardewvalley.apply(
    aplicar_multiplicador, axis=1)

In [94]:
dataframe_stardewvalley

,Nome_Fruto,Inverno,Outono,Primavera,Verão,Preco_Compra,Local_Compra,Preco_Venda_Comum,Preco_Venda_Incomum,Dias_Para_Amadurecer,Dias_Até_Colher_Novamente
0,Alho,0,0,1,0,40,Armazém,60,75,4,NaN
1,Arroz não moído,0,0,1,0,40,Armazém,30,37,8,NaN
2,Batata,0,0,1,0,50,Armazém,80,100,6,NaN
3,Chirívia,0,0,1,0,20,Armazém,35,43,4,NaN
4,Couve,0,0,1,0,70,Armazém,110,137,6,NaN
5,Couve-flor,0,0,1,0,80,Armazém,175,218,12,NaN
6,Grão de café,0,0,1,1,2500,Carrinho,60,72,10,2.0
7,Jasmim-Azul,0,0,1,0,30,Armazém,50,62,7,NaN
8,Morango,0,0,1,0,100,Festival,120,150,8,4.0
9,Ruibarbo,0,0,1,0,100,Oásis,220,275,13,NaN


## LEMBRAR

* Utilizar pipeline etl integrado com algum chat generativo para dar dicas.

* Maioria das sementes só crescem em determinada estação.

* Algumas sementes dão fruto apenas 1x, outros dão de n em n dias.

* De restrições, possuo: quantidade de ouro disponivel, quantidade de energia e quantidade de quadrados (unidades totais de semente).

* Algumas plantas cobrem quantidade diferente de quadrados.

* Existem niveis diferentes de qualidade de fruto, levar probabilidade de cada em conta.

* Alguns frutos só são liberados após algum tempo.

* Questionar se a única coisa a maximizar é o lucro < < < < < LUCRO != Retorno. Tem que levar em conta os gastos.

* Tbm tem diferenca de precos entre piere e jojo OK.

* Utilizar calculo de valor esperado. De acordo com as probabilidades de cada qualidade para cada fruta.

* Calculations do not take into account Fertilizer, crop quality, or the Tiller or Agriculturist Professions. Variations are noted in or below each table, if applicable. It is assumed that the crop is watered on the day of planting.

* The general formula is: Minimum Gold per Day = ((Max Harvests × Sell Price per Harvest) − Seed Price) / Growing Days.

* Growing Days = Days to Maturity + ((Max Harvests − 1) × Days to Regrow).

* Days to Maturity and Days to Regrow are listed in each table.

* Max Harvests is normally 1, but for crops that continue to produce, it is the actual number of harvests that can be obtained in the growing season(s).

* Sell Price per Harvest is normally the same as the Sell Price of one normal-quality crop. Extra crops are not counted since their occurrence is rare, except in the case of potatoes. The chance of an extra potato is ≈25%, much higher than with any other crop. Thus, the Sell Price per Harvest is 1.25 × 80 instead of 1 × 80.